In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd
import numpy as np
import csv
import fl_utils.fl_baseline_functions as base

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\fl-model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 3e-5, 1e-2, 5e-2]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### Lasso
c = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 3e-5, 1e-2, 5e-2]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### LinearSVM
c = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 3e-5, 1e-2, 5e-2]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### Random Forest
n_estimators =  [50,100,150,200,250]
depth = [1,2,3]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005, \
            0.006, 0.007, 0.008, 0.009, 0.01]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y,  
                     depth=depth, 
                     estimators=n_estimators, 
                     impurity=impurity,
                     seed=816)

#### XGBoost
learning_rate = [0.05]
depth = [1,2,3]
n_estimators = [40,60,80,100,150,200,250]
gamma = [6,8,10,12]
child_weight = [6,8,10,12]
subsample = [0.5]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y, 
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       gamma=gamma, 
                       child_weight=child_weight, 
                       subsample=subsample, 
                       seed=816)

#### save results
summary_general6_fl_model = {"Logistic": logistic_summary,
                             "Lasso": lasso_summary,
                             "LinearSVM": svm_summary,
                             "RF": rf_summary,
                             "XGBoost": xgb_summary}

In [4]:
results = []
for model_name, model_summary in summary_general6_fl_model.items():
    results.append([model_name, 
                    np.mean(model_summary['FL_validation']), 
                    np.mean(model_summary['auc_diff']), 
                    np.mean(model_summary['KY_score'])])
results

[['Logistic', 0.6224299193038751, 0.0287207509804591, 0.6561484336078338],
 ['Lasso', 0.6216592337259225, 0.022970654697237823, 0.6575000555330626],
 ['LinearSVM', 0.615150343149782, 0.032362645070650696, 0.6106837388223525],
 ['RF', 0.6188816959050987, 0.04992005796968506, 0.6870954299168256],
 ['XGBoost', 0.6270998435054937, 0.04589317079228743, 0.6695107187717296]]

In [5]:
path = "./results/baselines/six-month/"
results = [["", "Logistic", "Lasso", "Linear SVM", "Random Forest", "XGBoost"],
           ["General",             
            np.str(round(np.mean(logistic_summary['KY_score']),3)) + " (" + np.str(round(np.std(logistic_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(lasso_summary['KY_score']), 3)) + " (" + np.str(round(np.std(lasso_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(svm_summary['KY_score']),3)) + " (" + np.str(round(np.std(svm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(rf_summary['KY_score']),3)) + " (" + np.str(round(np.std(rf_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(xgb_summary['KY_score']),3)) + " (" + np.str(round(np.std(xgb_summary['KY_score']),3)) + ")"]]
with open(path + 'six-month-fl-baseline.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)